In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
ex1 = pd.read_parquet('회원_신용_청구.parquet')

In [3]:
ex1

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능_CA,소지카드수_이용가능_신용,입회일자_신용,회원여부_연체,동의여부_한도증액안내,...,포인트_포인트_건별_B0M,포인트_포인트_월적립_B0M,포인트_잔여포인트_B0M,마일_적립포인트_R12M,마일_잔여포인트_B0M,할인건수_R3M,할인건수_B0M,할인금액_B0M,선결제건수_R3M,연체건수_R3M
0,201807,TRAIN_000000,2,40,D,1,1,20130101,0,1,...,1444,0,0,0,0,1,1,0,0,0
1,201807,TRAIN_000001,1,30,E,1,1,20170801,0,0,...,0,0,0,0,0,1,1,0,0,0
2,201807,TRAIN_000002,1,30,C,1,1,20080401,0,1,...,2305,0,0,0,0,1,1,0,0,0
3,201807,TRAIN_000003,2,40,D,1,2,20160501,0,1,...,1452,0,0,0,0,1,1,0,0,0
4,201807,TRAIN_000004,2,40,E,1,1,20180601,0,0,...,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,-1,E,1,1,20010701,0,0,...,0,0,0,0,0,1,1,0,0,0
2399996,201812,TRAIN_399996,2,50,D,1,1,20170701,0,0,...,3339,0,0,0,0,1,1,0,0,0
2399997,201812,TRAIN_399997,1,30,C,1,1,20090501,0,0,...,0,0,0,0,0,1,1,0,0,0
2399998,201812,TRAIN_399998,1,40,E,1,1,20130101,0,0,...,0,0,0,0,0,1,1,0,0,0


In [10]:
X = ex1.drop(columns=['ID', '기준년월', 'Segment'])
X = X.select_dtypes(include=[np.number])
y = pd.factorize(ex1['Segment'], sort=True)[0]  # A~E → 0~4로 인코딩

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    class_weight='balanced',  # ← 클래스 불균형 보정
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("★ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

test_df = pd.read_parquet("회원_신용_청구_test.parquet")

X_submit = test_df.drop(columns=['ID', '기준년월'])
X_submit = X_submit.select_dtypes(include=[np.number])

y_submit_pred = rf.predict(X_submit)

segment_labels = pd.factorize(ex1['Segment'], sort=True)[1]  # Index([A, B, C, D, E], dtype=object)
predicted_segments = segment_labels[y_submit_pred]

predict = pd.DataFrame({
    'ID': test_df['ID'],
    'Segment': predicted_segments
})

print(predict.head())

predict.to_csv("rf_model.csv", index=False)

★ Accuracy: 0.7801472222222222
              precision    recall  f1-score   support

           0     0.1211    0.9656    0.2152       291
           1     0.7660    0.8372    0.8000        43
           2     0.4250    0.6474    0.5131     38277
           3     0.3876    0.6172    0.4762    104773
           4     0.9583    0.8185    0.8829    576616

    accuracy                         0.7801    720000
   macro avg     0.5316    0.7772    0.5775    720000
weighted avg     0.8465    0.7801    0.8038    720000

           ID Segment
0  TEST_00000       D
1  TEST_00001       D
2  TEST_00002       D
3  TEST_00003       E
4  TEST_00004       E


PermissionError: [Errno 13] Permission denied: 'rf_segment_prediction.csv'

In [12]:
predict.to_csv("rf_segment_prediction1.csv", index=False)

In [13]:

df_pred = pd.read_csv("rf_segment_prediction1.csv")  # ID, Segment

df_mode = df_pred.groupby("ID")["Segment"].agg(lambda x: x.mode().iloc[0]).reset_index()

# 3. sample_submission 포맷 확인
sample = pd.read_csv("sample_submission.csv")
print("샘플 행 수:", len(sample))  # 100,000

# 4. ID 기준으로 merge
final_submission = sample[['ID']].merge(df_mode, on='ID', how='left')

# 5. Segment가 없는 경우 예비값 지정 (예: 'A')
final_submission['Segment'] = final_submission['Segment'].fillna('A')

# 6. 저장
final_submission.to_csv("final_segment_submission.csv", index=False)


샘플 행 수: 100000
